In [2]:
val covid070320 = spark
    .read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv("03-07-2020.csv")

covid070320 = [Province/State: string, Country/Region: string ... 6 more fields]


lastException: Throwable = null


[Province/State: string, Country/Region: string ... 6 more fields]

In [4]:
covid070320.count()

225

In [10]:
covid070320.sort($"Deaths".desc).show()

+--------------------+--------------+-------------------+---------+------+---------+--------+---------+
|      Province/State|Country/Region|        Last Update|Confirmed|Deaths|Recovered|Latitude|Longitude|
+--------------------+--------------+-------------------+---------+------+---------+--------+---------+
|               Hubei|Mainland China|2020-03-07 11:13:04|    67666|  2959|    43500| 30.9756| 112.2707|
|                null|         Italy|2020-03-07 17:33:03|     5883|   233|      589|    43.0|     12.0|
|                null|          Iran|2020-03-07 17:43:05|     5823|   145|     1669|    32.0|     53.0|
|                null|   South Korea|2020-03-07 10:13:22|     7041|    44|      135|    36.0|    128.0|
|               Henan|Mainland China|2020-03-07 11:23:10|     1272|    22|     1244|  33.882|  113.614|
|     King County, WA|            US|2020-03-07 20:23:03|       71|    15|        1|  47.548|-121.9836|
|        Heilongjiang|Mainland China|2020-03-07 09:03:05|      4

In [11]:
import org.apache.spark.sql.functions._
covid070320.sort(desc("Deaths")).show()

+--------------------+--------------+-------------------+---------+------+---------+--------+---------+
|      Province/State|Country/Region|        Last Update|Confirmed|Deaths|Recovered|Latitude|Longitude|
+--------------------+--------------+-------------------+---------+------+---------+--------+---------+
|               Hubei|Mainland China|2020-03-07 11:13:04|    67666|  2959|    43500| 30.9756| 112.2707|
|                null|         Italy|2020-03-07 17:33:03|     5883|   233|      589|    43.0|     12.0|
|                null|          Iran|2020-03-07 17:43:05|     5823|   145|     1669|    32.0|     53.0|
|                null|   South Korea|2020-03-07 10:13:22|     7041|    44|      135|    36.0|    128.0|
|               Henan|Mainland China|2020-03-07 11:23:10|     1272|    22|     1244|  33.882|  113.614|
|     King County, WA|            US|2020-03-07 20:23:03|       71|    15|        1|  47.548|-121.9836|
|        Heilongjiang|Mainland China|2020-03-07 09:03:05|      4

In [12]:
covid070320.sort(desc("Deaths")).explain()

== Physical Plan ==
*(2) Sort [Deaths#14 DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(Deaths#14 DESC NULLS LAST, 200)
   +- *(1) FileScan csv [Province/State#10,Country/Region#11,Last Update#12,Confirmed#13,Deaths#14,Recovered#15,Latitude#16,Longitude#17] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/pphetra/projects/sparktest/03-07-2020.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Province/State:string,Country/Region:string,Last Update:timestamp,Confirmed:int,Deaths:int...


In [13]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [14]:
covid070320.sort(desc("Deaths")).explain()

== Physical Plan ==
*(2) Sort [Deaths#14 DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(Deaths#14 DESC NULLS LAST, 5)
   +- *(1) FileScan csv [Province/State#10,Country/Region#11,Last Update#12,Confirmed#13,Deaths#14,Recovered#15,Latitude#16,Longitude#17] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/pphetra/projects/sparktest/03-07-2020.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Province/State:string,Country/Region:string,Last Update:timestamp,Confirmed:int,Deaths:int...


In [19]:
covid070320.groupBy($"Country/Region").agg(sum("Deaths").as("Deaths")).sort(desc("Deaths")).show()

+--------------+------+
|Country/Region|Deaths|
+--------------+------+
|Mainland China|  3070|
|         Italy|   233|
|          Iran|   145|
|   South Korea|    44|
|            US|    17|
|        France|    11|
|         Spain|    10|
|         Japan|     6|
|        Others|     6|
|          Iraq|     4|
|            UK|     2|
|     Australia|     2|
|     Hong Kong|     2|
|   Philippines|     1|
|        Taiwan|     1|
|      Thailand|     1|
|    San Marino|     1|
|   Switzerland|     1|
|   Netherlands|     1|
|      Cameroon|     0|
+--------------+------+
only showing top 20 rows



In [20]:
covid070320.groupBy($"Country/Region")
    .agg(
        sum("Deaths").as("Deaths"),
        sum("Confirmed").as("Confirmed")
    )
    .sort(desc("Deaths")).show()

+--------------+------+---------+
|Country/Region|Deaths|Confirmed|
+--------------+------+---------+
|Mainland China|  3070|    80652|
|         Italy|   233|     5883|
|          Iran|   145|     5823|
|   South Korea|    44|     7041|
|            US|    17|      417|
|        France|    11|      949|
|         Spain|    10|      500|
|         Japan|     6|      461|
|        Others|     6|      696|
|          Iraq|     4|       54|
|            UK|     2|      206|
|     Australia|     2|       63|
|     Hong Kong|     2|      108|
|   Philippines|     1|        6|
|   Switzerland|     1|      268|
|      Thailand|     1|       50|
|        Taiwan|     1|       45|
|    San Marino|     1|       23|
|   Netherlands|     1|      188|
|        Bhutan|     0|        1|
+--------------+------+---------+
only showing top 20 rows



In [24]:
covid070320.groupBy($"Country/Region")
    .agg(
        sum("Deaths").as("Deaths"),
        sum("Confirmed").as("Confirmed")
    )
    .withColumn("fatality", $"Deaths" * 100 / $"Confirmed")
    .sort(desc("Deaths")).show()

+--------------+------+---------+------------------+
|Country/Region|Deaths|Confirmed|          fatality|
+--------------+------+---------+------------------+
|Mainland China|  3070|    80652|3.8064772107325298|
|         Italy|   233|     5883|3.9605643379228286|
|          Iran|   145|     5823|2.4901253649321657|
|   South Korea|    44|     7041|0.6249112341996875|
|            US|    17|      417|  4.07673860911271|
|        France|    11|      949|1.1591148577449948|
|         Spain|    10|      500|               2.0|
|         Japan|     6|      461|1.3015184381778742|
|        Others|     6|      696|0.8620689655172413|
|          Iraq|     4|       54| 7.407407407407407|
|     Hong Kong|     2|      108|1.8518518518518519|
|     Australia|     2|       63|3.1746031746031744|
|            UK|     2|      206| 0.970873786407767|
|   Netherlands|     1|      188|0.5319148936170213|
|    San Marino|     1|       23|4.3478260869565215|
|        Taiwan|     1|       45|2.22222222222

In [23]:
import org.apache.spark.sql.functions.format_number
covid070320.groupBy($"Country/Region")
    .agg(
        sum("Deaths").as("Deaths"),
        sum("Confirmed").as("Confirmed")
    )
    .withColumn("fatality", format_number($"Deaths" * 100 / $"Confirmed", 2))
    .sort(desc("Deaths")).show()

+--------------+------+---------+--------+
|Country/Region|Deaths|Confirmed|fatality|
+--------------+------+---------+--------+
|Mainland China|  3070|    80652|    3.81|
|         Italy|   233|     5883|    3.96|
|          Iran|   145|     5823|    2.49|
|   South Korea|    44|     7041|    0.62|
|            US|    17|      417|    4.08|
|        France|    11|      949|    1.16|
|         Spain|    10|      500|    2.00|
|        Others|     6|      696|    0.86|
|         Japan|     6|      461|    1.30|
|          Iraq|     4|       54|    7.41|
|     Australia|     2|       63|    3.17|
|     Hong Kong|     2|      108|    1.85|
|            UK|     2|      206|    0.97|
|    San Marino|     1|       23|    4.35|
|   Switzerland|     1|      268|    0.37|
|      Thailand|     1|       50|    2.00|
|   Netherlands|     1|      188|    0.53|
|        Taiwan|     1|       45|    2.22|
|   Philippines|     1|        6|   16.67|
|          Oman|     0|       16|    0.00|
+----------